### TODO Query 1: Number of: COVID cases, Opened and Closed Businesses, and Crime Events grouped by month and year

### Query 1A: Number of COVID cases and deaths grouped by month and year

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (SUM(?newCases) AS ?numOfCovidCases) (ROUND((?numOfCovidCases*100 / 30))/100 AS ?casesRatio) (SUM(?newDeaths) AS ?numOfDeaths) (ROUND((?numOfDeaths*100 / 30))/100 AS ?deathsRatio)
WHERE {
    ?day lao:hasDate ?date ;
         lao:hasNewCases ?newCases ;
         lao:hasNewDeaths ?newDeaths .
         
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)

### Query 1B: Number of Opened Businesses grouped by month and year

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (COUNT(?openBusiness) AS ?openedBusinesses) (ROUND((?openedBusinesses*100 / 30))/100 AS ?ratio)
WHERE {
    ?openBusiness lao:openedOnDate ?day .
    ?day lao:hasDate ?date .
    
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)

### Query 1C: Number of Closed Businesses grouped by month and year

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (COUNT(?closeBusiness) AS ?closedBusinesses) (ROUND(?closedBusinesses*100 / 30)/100 AS ?ratio)
WHERE {
    ?closeBusiness lao:closedOnDate ?day .
    ?day lao:hasDate ?date .
    
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)

### Query 1D: Number of Crime Events grouped by month and year

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?period (COUNT(?crimeEvent) AS ?crimeEvents) (ROUND(?crimeEvents*100 / 30)/100 AS ?ratio)
WHERE {
    ?crimeEvent lao:occurredOnDate ?day .
    ?day lao:hasDate ?date .
    
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?period
ORDER BY ASC(?period)

### Query 2A: Number of COVID Cases and Deaths grouped by period, with ratio on period duration

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (SUM(?newCases) AS ?numOfCovidCases) (ROUND((?numOfCovidCases*100 / ?duration))/100 AS ?casesRatio) (SUM(?newDeaths) AS ?numOfDeaths) (ROUND((?numOfDeaths*100 / ?duration))/100 AS ?deathsRatio)
WHERE {
    ?day lao:hasDate ?date ;
         lao:hasNewCases ?newCases ;
         lao:hasNewDeaths ?newDeaths ;
         lao:belongsTo ?period .
         
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed 

### Query 2B: Number of Opened Businesses grouped by period, with ratio on period duration

In [ ]:
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (COUNT(?openBusiness) AS ?openedBusinesses) (ROUND(?openedBusinesses*100/?duration)/100 AS ?openedRatio)
WHERE {
    ?openBusiness lao:openedOnDate ?day .
    ?day lao:hasDate ?date ;
         lao:belongsTo ?period .
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed 

### Query 2C: Number of Closed Businesses grouped by period, with ratio on period duration

In [ ]:
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (COUNT(?closeBusiness) AS ?closedBusinesses) (ROUND(?closedBusinesses*100/?duration)/100 AS ?closedRatio)
WHERE {
    ?closeBusiness lao:closedOnDate ?day .
    ?day lao:hasDate ?date ;
         lao:belongsTo ?period .
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed 

### Query 2D: Number of Crime Events grouped by period, with ratio on period duration

In [ ]:
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX fn: <http://www.w3.org/2005/xpath-functions#>

SELECT ?periodLabel ?startDateParsed ?endDateParsed ?duration (COUNT(?crimeEvent) AS ?crimeEvents) (ROUND(?crimeEvents*100/?duration)/100 AS ?crimeEventsRatio)
WHERE {
    ?crimeEvent lao:occurredOnDate ?day .
    ?day lao:hasDate ?date ;
         lao:belongsTo ?period .
    ?period lao:hasLabel ?periodLabel ;
            lao:hasStartDate ?startDate ;
            lao:hasEndDate ?endDate ;
            lao:hasDuration ?duration .
         
    FILTER(?date >= ?startDate && ?date <= ?endDate)
    BIND(SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed)
    BIND(SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed)
}
GROUP BY ?periodLabel ?startDateParsed ?endDateParsed ?duration
ORDER BY ?startDateParsed 

### Query 3A: Number of Opened Businesses grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?openBusiness) AS ?openedBusinesses) (((ROUND(?openedBusinesses * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?openBusiness lao:openedOnDate ?day ;
                  lao:locatedIn ?location .
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface
ORDER BY DESC(?ratio)

### Query 3B: Number of Closed Businesses grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?closeBusiness) AS ?closedBusinesses) (((ROUND(?closedBusinesses * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?closeBusiness lao:closedOnDate ?day ;
                  lao:locatedIn ?location .
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface
ORDER BY DESC(?ratio)

### Query 3C: Number of Crime Events grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location .
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface
ORDER BY DESC(?ratio)

### Query 3D: Number of Violent Crime Events grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:ViolentCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)

### Query 3E: Number of Sexual Crime Events grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:SexualCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)

### Query 3F: Number of Property Crime Events grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:PropertyCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)

### Query 3G: Number of White Collar Crime Events grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:WhiteCollarCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)

### Query 3H: Number of Public Order Crime Events grouped by area, with ratio on area surface

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?areaName ?areaAcronym (ROUND(?areaSurface*100)/100 AS ?areaSize) (COUNT(?crimeEvent) AS ?crimeEvents) (((ROUND(?crimeEvents * 1000 / ?areaSurface)) / 1000) AS ?ratio) 
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
                  lao:occurredInLocation ?location ;
                  lao:isOfType ?crimeCat .
    ?crimeCat a lao:PublicOrderCrime . 
    ?day lao:hasDate ?date .
    ?location lao:belongsToArea ?area .
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym ;
          lao:areaSurface ?areaSurface
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
}
GROUP BY ?areaName ?areaAcronym ?areaSurface ?crimeCat
ORDER BY DESC(?ratio)

### Query 4A: Number of Opened Businesses grouped by NAICS code

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?naicsDesc (COUNT(?openBusiness) AS ?openedBusinesses) (ROUND(?openedBusinesses*1000 / ?totBusinesses)/1000 AS ?ratio) 
WHERE {
    ?openBusiness lao:openedOnDate ?day ;
                  lao:hasNaics ?naics .
    ?day lao:hasDate ?date .
    ?naics lao:naicsCode ?naicsCode ;
           lao:naicsDescription ?naicsDesc .
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)

    {
        SELECT DISTINCT (COUNT(?openBus) AS ?totBusinesses) 
        WHERE {
            ?openBus lao:openedOnDate ?openDay .
            ?openDay lao:hasDate ?openDate .
            FILTER (xsd:date(?openDate) >= "2020-01-01"^^xsd:date && xsd:date(?openDate) <= "2022-12-31"^^xsd:date)
        }                        
    }
}
GROUP BY ?naicsCode ?naicsDesc ?totBusinesses
ORDER BY DESC(?openedBusinesses)

### Query 4B: Number of Closed Businesses grouped by NAICS code

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT DISTINCT ?naicsDesc (COUNT(?closeBusiness) AS ?closedBusinesses) (ROUND(?closedBusinesses*1000 / ?totBusinesses)/1000 AS ?ratio) 
WHERE {
    ?closeBusiness lao:closedOnDate ?day ;
                  lao:hasNaics ?naics .
    ?day lao:hasDate ?date .
    ?naics lao:naicsCode ?naicsCode ;
           lao:naicsDescription ?naicsDesc .
    # convert to strings
    BIND(STR(YEAR(?date)) AS ?year)
    BIND(STR(MONTH(?date)) AS ?month)
    # pad with zeros
    BIND(CONCAT("00", ?year) AS ?paddedYear)
    BIND(CONCAT("0000", ?month) AS ?paddedMonth)
    # extract the right number of digits from the padded strings
    BIND(SUBSTR(?paddedYear, STRLEN(?paddedYear)-3) AS ?fourDigitYear)
    BIND(SUBSTR(?paddedMonth, STRLEN(?paddedMonth)-1) AS ?twoDigitMonth)
    # put it all back together
    BIND(CONCAT(?fourDigitYear, "-", ?twoDigitMonth) AS ?period)
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)

    {
        SELECT DISTINCT (COUNT(?closeBus) AS ?totBusinesses) 
        WHERE {
            ?closeBus lao:closedOnDate ?closeDay .
            ?closeDay lao:hasDate ?closeDate .		
            FILTER (xsd:date(?closeDate) >= "2020-01-01"^^xsd:date && xsd:date(?closeDate) <= "2022-12-31"^^xsd:date)
        }                        
    }
}
GROUP BY ?naicsCode ?naicsDesc ?totBusinesses
ORDER BY DESC(?closedBusinesses)

### Query 5A: Number of Crime Events grouped by crime category

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?crimeCategory (COUNT(?crimeEvent) AS ?numOfCrimeEvents) 
WHERE {
    ?crimeEvent lao:isOfType ?crime .
    ?crime a ?crimeCat .
    ?crimeCat rdfs:subClassOf lao:Crime .
    BIND(REPLACE(STR(?crimeCat), ".*/", "") AS ?crimeCategory)
}
GROUP BY(?crimeCategory)
ORDER BY DESC(?numOfCrimeEvents)

### Query 5B: Number of Crime Events grouped by period and crime category

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?periodLabel 
       (SUBSTR(STR(?startDate), 1, 10) AS ?startDateParsed) 
       (SUBSTR(STR(?endDate), 1, 10) AS ?endDateParsed) 
       (COUNT(?sexualCrimeEvent) AS ?numSexualCrimes)
       (COUNT(?propertyCrimeEvent) AS ?numPropertyCrimes)
       (COUNT(?whiteCollarCrimeEvent) AS ?numWhiteCollarCrimes)
       (COUNT(?violentCrimeEvent) AS ?numViolentCrimes)
       (COUNT(?publicOrderCrimeEvent) AS ?numPublicOrderCrimes)
WHERE {
  ?crimeEvent lao:occurredOnDate ?day ;
              lao:isOfType ?crime .
  ?day lao:hasDate ?date ;
       lao:belongsTo ?period .
  ?period lao:hasLabel ?periodLabel ;
          lao:hasStartDate ?startDate ;
          lao:hasEndDate ?endDate.

  FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)

  OPTIONAL { ?crime a lao:SexualCrime . BIND(?crimeEvent AS ?sexualCrimeEvent) }
  OPTIONAL { ?crime a lao:PropertyCrime . BIND(?crimeEvent AS ?propertyCrimeEvent) }
  OPTIONAL { ?crime a lao:WhiteCollarCrime . BIND(?crimeEvent AS ?whiteCollarCrimeEvent) }
  OPTIONAL { ?crime a lao:ViolentCrime . BIND(?crimeEvent AS ?violentCrimeEvent) }
  OPTIONAL { ?crime a lao:PublicOrderCrime . BIND(?crimeEvent AS ?publicOrderCrimeEvent) }
}
GROUP BY ?periodLabel ?startDate ?endDate
ORDER BY (?startDate)


### Query 5C: Number of Crime Events grouped by area and crime category

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT 
?areaName ?areaAcronym 
(COUNT(?crimeEvent) AS ?totNumOfCrimes)
(COUNT(?sexualCrimeEvent) AS ?numSexualCrimes)
(COUNT(?propertyCrimeEvent) AS ?numPropertyCrimes)
(COUNT(?whiteCollarCrimeEvent) AS ?numWhiteCollarCrimes)
(COUNT(?violentCrimeEvent) AS ?numViolentCrimes)
(COUNT(?publicOrderCrimeEvent) AS ?numPublicOrderCrimes)
WHERE {
    ?crimeEvent lao:occurredOnDate ?day ;
              lao:isOfType ?crime ;
              lao:occurredInLocation ?location .
  
    ?location lao:belongsToArea ?area .
    
    ?area lao:areaName ?areaName ;
          lao:areaAcronym ?areaAcronym .
    
    ?day lao:hasDate ?date ;
       lao:belongsTo ?period .
  
    ?period lao:hasLabel ?periodLabel ;
          lao:hasStartDate ?startDate ;
          lao:hasEndDate ?endDate.
    
    FILTER (xsd:date(?date) >= "2020-01-01"^^xsd:date && xsd:date(?date) <= "2022-12-31"^^xsd:date)
  
    OPTIONAL { ?crime a lao:SexualCrime . BIND(?crimeEvent AS ?sexualCrimeEvent) }
    OPTIONAL { ?crime a lao:PropertyCrime . BIND(?crimeEvent AS ?propertyCrimeEvent) }
    OPTIONAL { ?crime a lao:WhiteCollarCrime . BIND(?crimeEvent AS ?whiteCollarCrimeEvent) }
    OPTIONAL { ?crime a lao:ViolentCrime . BIND(?crimeEvent AS ?violentCrimeEvent) }
    OPTIONAL { ?crime a lao:PublicOrderCrime . BIND(?crimeEvent AS ?publicOrderCrimeEvent) }
}
GROUP BY ?areaName ?areaAcronym
ORDER BY DESC(?totNumOfCrimes)

## Query 6
### Get the number of crimes (for each period), grouped by the weapon used

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>

SELECT ?moDesc
       (SUM(?crimeCovidStartingPeriod) AS ?crimeCovidStartingPeriod)
       (SUM(?crimeFirstLockdownPeriod) AS ?crimeFirstLockdownPeriod)
       (SUM(?crimeFirstReopeningPeriod) AS ?crimeFirstReopeningPeriod)
       (SUM(?crimeMaskMandatePeriod) AS ?crimeMaskMandatePeriod)
       (SUM(?crimeRestrictionsEasedPeriod) AS ?crimeRestrictionsEasedPeriod)
       (SUM(?crimeVaccineAvailabilityPeriod) AS ?crimeVaccineAvailabilityPeriod)
       (SUM(?crimeRestrictionsDroppedPeriod) AS ?crimeRestrictionsDroppedPeriod)
       (SUM(?crimeVaccineRequirementPeriod) AS ?crimeVaccineRequirementPeriod)
       (SUM(?crimeMaskMandateExtendedPeriod) AS ?crimeMaskMandateExtendedPeriod)
       (SUM(?crimeSecondRestrictionsEasedPeriod) AS ?crimeSecondRestrictionsEasedPeriod)
       (SUM(?crimeExecutiveOrdersDroppedPeriod) AS ?crimeExecutiveOrdersDroppedPeriod)
WHERE {
  ?mo lao:moDesc ?moDesc .
  ?crimeEvent lao:hasModusOperandi ?mo ;
              lao:occurredOnDate ?day .
  ?day lao:belongsTo ?period .
  ?period lao:hasLabel ?label .

  BIND(IF(?label = "Covid Starting Period", 1, 0) AS ?crimeCovidStartingPeriod)
  BIND(IF(?label = "First Lockdown Period", 1, 0) AS ?crimeFirstLockdownPeriod)
  BIND(IF(?label = "First Reopening Period", 1, 0) AS ?crimeFirstReopeningPeriod)
  BIND(IF(?label = "Mask Mandate Period", 1, 0) AS ?crimeMaskMandatePeriod)
  BIND(IF(?label = "Restrictions Eased Period", 1, 0) AS ?crimeRestrictionsEasedPeriod)
  BIND(IF(?label = "Vaccine Availability Period", 1, 0) AS ?crimeVaccineAvailabilityPeriod)
  BIND(IF(?label = "Restrictions Dropped Period", 1, 0) AS ?crimeRestrictionsDroppedPeriod)
  BIND(IF(?label = "Vaccine Requirement Period", 1, 0) AS ?crimeVaccineRequirementPeriod)
  BIND(IF(?label = "Mask Mandate Extended Period", 1, 0) AS ?crimeMaskMandateExtendedPeriod)
  BIND(IF(?label = "Second Restrictions Eased Period", 1, 0) AS ?crimeSecondRestrictionsEasedPeriod)
  BIND(IF(?label = "Executive Orders Dropped Period", 1, 0) AS ?crimeExecutiveOrdersDroppedPeriod)
}
GROUP BY ?moDesc
ORDER BY DESC(?crimeCovidStartingPeriod)

## Query 7
### Get the number of crimes (for each period), grouped by the premises occurred

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>

SELECT ?premisDesc
       (SUM(?crimeCovidStartingPeriod) AS ?crimeCovidStartingPeriod)
       (SUM(?crimeFirstLockdownPeriod) AS ?crimeFirstLockdownPeriod)
       (SUM(?crimeFirstReopeningPeriod) AS ?crimeFirstReopeningPeriod)
       (SUM(?crimeMaskMandatePeriod) AS ?crimeMaskMandatePeriod)
       (SUM(?crimeRestrictionsEasedPeriod) AS ?crimeRestrictionsEasedPeriod)
       (SUM(?crimeVaccineAvailabilityPeriod) AS ?crimeVaccineAvailabilityPeriod)
       (SUM(?crimeRestrictionsDroppedPeriod) AS ?crimeRestrictionsDroppedPeriod)
       (SUM(?crimeVaccineRequirementPeriod) AS ?crimeVaccineRequirementPeriod)
       (SUM(?crimeMaskMandateExtendedPeriod) AS ?crimeMaskMandateExtendedPeriod)
       (SUM(?crimeSecondRestrictionsEasedPeriod) AS ?crimeSecondRestrictionsEasedPeriod)
       (SUM(?crimeExecutiveOrdersDroppedPeriod) AS ?crimeExecutiveOrdersDroppedPeriod)
WHERE {
  ?premis lao:premisDesc ?premisDesc .
  ?crimeEvent lao:hasPremis ?premis ;
              lao:occurredOnDate ?day .
  ?day lao:belongsTo ?period .
  ?period lao:hasLabel ?label .

  BIND(IF(?label = "Covid Starting Period", 1, 0) AS ?crimeCovidStartingPeriod)
  BIND(IF(?label = "First Lockdown Period", 1, 0) AS ?crimeFirstLockdownPeriod)
  BIND(IF(?label = "First Reopening Period", 1, 0) AS ?crimeFirstReopeningPeriod)
  BIND(IF(?label = "Mask Mandate Period", 1, 0) AS ?crimeMaskMandatePeriod)
  BIND(IF(?label = "Restrictions Eased Period", 1, 0) AS ?crimeRestrictionsEasedPeriod)
  BIND(IF(?label = "Vaccine Availability Period", 1, 0) AS ?crimeVaccineAvailabilityPeriod)
  BIND(IF(?label = "Restrictions Dropped Period", 1, 0) AS ?crimeRestrictionsDroppedPeriod)
  BIND(IF(?label = "Vaccine Requirement Period", 1, 0) AS ?crimeVaccineRequirementPeriod)
  BIND(IF(?label = "Mask Mandate Extended Period", 1, 0) AS ?crimeMaskMandateExtendedPeriod)
  BIND(IF(?label = "Second Restrictions Eased Period", 1, 0) AS ?crimeSecondRestrictionsEasedPeriod)
  BIND(IF(?label = "Executive Orders Dropped Period", 1, 0) AS ?crimeExecutiveOrdersDroppedPeriod)
}
GROUP BY ?premisDesc
ORDER BY DESC(?crimeCovidStartingPeriod)

## Query 8
### Get the number of crimes (for each period), grouped by the modus operandi involved

In [ ]:
PREFIX lao: <http://www.bitsei.it/losAngelesOntology/>

SELECT ?moDesc
       (SUM(?crimeCovidStartingPeriod) AS ?crimeCovidStartingPeriod)
       (SUM(?crimeFirstLockdownPeriod) AS ?crimeFirstLockdownPeriod)
       (SUM(?crimeFirstReopeningPeriod) AS ?crimeFirstReopeningPeriod)
       (SUM(?crimeMaskMandatePeriod) AS ?crimeMaskMandatePeriod)
       (SUM(?crimeRestrictionsEasedPeriod) AS ?crimeRestrictionsEasedPeriod)
       (SUM(?crimeVaccineAvailabilityPeriod) AS ?crimeVaccineAvailabilityPeriod)
       (SUM(?crimeRestrictionsDroppedPeriod) AS ?crimeRestrictionsDroppedPeriod)
       (SUM(?crimeVaccineRequirementPeriod) AS ?crimeVaccineRequirementPeriod)
       (SUM(?crimeMaskMandateExtendedPeriod) AS ?crimeMaskMandateExtendedPeriod)
       (SUM(?crimeSecondRestrictionsEasedPeriod) AS ?crimeSecondRestrictionsEasedPeriod)
       (SUM(?crimeExecutiveOrdersDroppedPeriod) AS ?crimeExecutiveOrdersDroppedPeriod)
WHERE {
  ?mo lao:moDesc ?moDesc .
  ?crimeEvent lao:hasModusOperandi ?mo ;
              lao:occurredOnDate ?day .
  ?day lao:belongsTo ?period .
  ?period lao:hasLabel ?label .

  BIND(IF(?label = "Covid Starting Period", 1, 0) AS ?crimeCovidStartingPeriod)
  BIND(IF(?label = "First Lockdown Period", 1, 0) AS ?crimeFirstLockdownPeriod)
  BIND(IF(?label = "First Reopening Period", 1, 0) AS ?crimeFirstReopeningPeriod)
  BIND(IF(?label = "Mask Mandate Period", 1, 0) AS ?crimeMaskMandatePeriod)
  BIND(IF(?label = "Restrictions Eased Period", 1, 0) AS ?crimeRestrictionsEasedPeriod)
  BIND(IF(?label = "Vaccine Availability Period", 1, 0) AS ?crimeVaccineAvailabilityPeriod)
  BIND(IF(?label = "Restrictions Dropped Period", 1, 0) AS ?crimeRestrictionsDroppedPeriod)
  BIND(IF(?label = "Vaccine Requirement Period", 1, 0) AS ?crimeVaccineRequirementPeriod)
  BIND(IF(?label = "Mask Mandate Extended Period", 1, 0) AS ?crimeMaskMandateExtendedPeriod)
  BIND(IF(?label = "Second Restrictions Eased Period", 1, 0) AS ?crimeSecondRestrictionsEasedPeriod)
  BIND(IF(?label = "Executive Orders Dropped Period", 1, 0) AS ?crimeExecutiveOrdersDroppedPeriod)
}
GROUP BY ?moDesc
ORDER BY DESC(?crimeCovidStartingPeriod)